# Data Cleaning (preparation)

In [ ]:
!pip install pandas rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 41.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd

df = pd.read_csv('DAT.csv', sep=';')

# Remove those units are not 'nM'
df_new = df[df['Standard Units'] == 'nM'].copy()

# Ensure the values are numeric
df_new['Standard Value'] = pd.to_numeric(df['Standard Value'], errors = 'coerce')
df_new = df.dropna(subset=['Standard Value','Smiles'])

print(f"有效數據量:{len(df_new)} 筆")

In [ ]:
from rdkit import Chem
from rdkit.Chem.SaltRemover import SaltRemover

def clean_salts(smiles_list):
  # Initialise SaltRemover
  remover = SaltRemover()
  cleaned_smiles = []

  for smi in smiles_list:
    try:
      mol = Chem.MolFromSmiles(smi)
      if mol:
        # 移除鹽類
        res = remover.StripMol(mol)
        # 轉回SMILES字串
        cleaned_smiles.append(Chem.MolToSmiles(res))

      else:
        cleaned_smiles.append(None)

    except:
      cleaned_smiles.append(None)

  return cleaned_smiles

df_new['Smiles'] = clean_salts(df_new['Smiles'])
df_new = df_new.dropna(subset=['Smiles'])

print(len(df_new))

In [ ]:
# Check if the 'Smiles' is multiple
df_new_unique = df_new.groupby('Smiles').agg({
    'Standard Value':'median', #活性取中位數
    'Molecule ChEMBL ID':'first' #ID取第一個代表
}).reset_index()

print(f"去重複後數據量:{len(df_new_unique)}筆")

In [ ]:
# Generate the label of active/inactive [Classification]
threshold = 1000
df_new_unique['Label'] = (df_new_unique['Standard Value'] <= threshold).astype(int)
df_new_unique.to_csv('DAT_Cleaned_Labeled.csv', index=False)

# Final Results
print('\n標籤分布情況')
print(df_new_unique['Label'].value_counts())

In [ ]:
# Sanity Check: Methylphenidate

target_id = 'CHEMBL796'

drug_row = df_new_unique[df_new_unique['Molecule ChEMBL ID'] == target_id]

if not drug_row.empty:
  print("✅ 找到 Methylphenidate 了！詳細數據如下：")
  print("-" * 50)
  print(f"ChEMBL ID:      {drug_row['Molecule ChEMBL ID'].values[0]}")
  print(f"SMILES 結構:    {drug_row['Smiles'].values[0]}")
  print(f"IC50 (nM):      {drug_row['Standard Value'].values[0]}")
  print(f"標籤 (Label):   {drug_row['Label'].values[0]}")
  print("-" * 50)

  ic50 = drug_row['Standard Value'].values[0]
  label = drug_row['Label'].values[0]

  if label == 1 and ic50 < 1000:
    print("Conclusion: the data is reasonable. Drug is correctly labelled as active.")

  else:
    print("Warning! Weird")

else:
  print("Warning! No CHEMBL 796 in this dataset!")


# Feature Engineering - FingerPrint

In [ ]:
from rdkit.Chem import AllChem
import numpy as np
from rdkit.Chem import rdFingerprintGenerator

df = pd.read_csv('DAT_Cleaned_Labeled.csv')

# fpSize = 2048 (Fingerprint length)
mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)

def smiles_to_fp(smiles, radius=2, n_bits = 2048):
    """
    將SMILES轉換為Morgan Fingerprint(位元陣列)
    radius = 2 相當於ECFP4
    [4 in ECFP4 corresponds to the diameter of the atom environments considered,
    while the Morgan fingerprints take a radius parameter.
    So the examples above, with radius=2, are roughly equivalent to ECFP4 and FCFP4.]
    from rdkit.org/
    """

    try:
      mol = Chem.MolFromSmiles(smiles)
      if mol:
        # 生成指紋物件
        fp = mfpgen.GetFingerprint(mol)
        # 轉換為Numpy陣列
        arr = np.zeros((1,))
        Chem.DataStructs.ConvertToNumpyArray(fp, arr)
        return arr

      else:
       return None

    except:
      return None

print("正在生成 Morgan Fingerprints (2048 bit)...")

# 執行轉換
fps = [smiles_to_fp(s) for s in df['Smiles']]

# 每一行是一個分子、每一列是一個特徵(Bit)
X = np.array(fps)
y = df['Label'].values

print(f"特徵矩陣形狀X shape: {X.shape}")
print(f"標籤形狀y shape: {y.shape}")


In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.7 MB/s eta 0:00:00


# Feature Engineering - Graph Data for the future GNN use

In [ ]:
import torch
from torch_geometric.data import Data, DataLoader

# 1. 原子特徵提取函數
def get_atom_features(atom):

  # Extract 原子序數,雜化軌域, 是否在環上
  features = [
      atom.GetAtomicNum(),
      atom.GetChiralTag(),
      atom.GetTotalDegree(),
      atom.GetFormalCharge(),
      atom.GetTotalNumHs(),
      atom.GetIsAromatic(),
      atom.GetHybridization()
  ]

  return np.array(features, dtype = np.float32)

# 2. SMILES --> Graph
def smiles_to_graph(smiles, label):
  mol = Chem.MolFromSmiles(smiles)
  if not mol:
    return None

  # 節點特徵(Node Features)
  atom_features = []
  for atom in mol.GetAtoms():
    atom_features.append(get_atom_features(atom))

  x = torch.tensor(atom_features, dtype = torch.float)

  # 邊緣特徵(Edge Features / Adjacency)
  edge_indices = []
  for bond in mol.GetBonds():
    i = bond.GetBeginAtomIdx()
    j = bond.GetEndAtomIdx()
    # 無向圖: 需要雙向連接
    edge_indices += [[i, j], [j,i]]

  # 如果分子只有一個原子(沒有鍵)，處理edge_index
  if len(edge_indices)>0:
    edge_index = torch.tensor(edge_indices, dtype = torch.long).t().contiguous()

  else:
    edge_index = torch.empty((2,0), dtype = torch.long)

  # 標籤(y)
  y = torch.tensor([label], dtype = torch.long)

  # 封裝成PyG Data 物件
  return Data(x = x, edge_index = edge_index, y = y)


In [ ]:
# 3. Read Data and Convert
df = pd.read_csv('DAT_Cleaned_Labeled.csv')
data_list = []

print("正在將SMILES轉換為分子圖數據")

for _, row in df.iterrows():
  graph = smiles_to_graph(row['Smiles'], row['Label'])
  if graph:
    data_list.append(graph)

print(f"✅ 成功轉換{len(data_list)} 個分子圖")

# 4.準備DataLoader (供GNN)
loader = DataLoader(data_list, batch_size=32, shuffle = True)

# Test: 查看第一個Batch
for batch in loader:
  print("\nBatch範例")
  print(batch)
  print(f"該Batch分子總原子數: {batch.num_nodes}")
  break

正在將SMILES轉換為分子圖數據


/tmp/ipython-input-2841401233.py:31: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  x = torch.tensor(atom_features, dtype = torch.float)


✅ 成功轉換2361 個分子圖

Batch範例
DataBatch(x=[745, 7], edge_index=[2, 1632], y=[32], batch=[745], ptr=[33])
該Batch分子總原子數: 745


/tmp/ipython-input-1942535130.py:15: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  loader = DataLoader(data_list, batch_size=32, shuffle = True)


#Training Phase: RF/XGBoost/GNN

In [ ]:
from sklearn.model_selection import train_test_split

# For Fingerprints(RF/XGBoost)
# X: 2048-bit array, y:Label

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state=42)

# For GNN (graphs)
# data_list is the input data
train_data, test_data = train_test_split(data_list, test_size = 0.2, random_state =42)
train_loader = DataLoader(train_data, batch_size = 32, shuffle =True)
test_loader = DataLoader(test_data, batch_size =32, shuffle = False)


/tmp/ipython-input-1068801127.py:11: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  train_loader = DataLoader(train_data, batch_size = 32, shuffle =True)
/tmp/ipython-input-1068801127.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  test_loader = DataLoader(test_data, batch_size =32, shuffle = False)


In [ ]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool

class GCN(torch.nn.Module):
  def __init__(self, num_node_features):
    super(GCN, self).__init__()
    self.conv1 = GCNConv(num_node_features, 64)
    self.conv2 = GCNConv(64,64)
    self.fc = torch.nn.Linear(64,2)
    # 分2類(0,1)

  def forward(self,data):
    x, edge_index, batch = data.x, data.edge_index, data.batch

    # 1. 圖卷積層
    x = self.conv1(x, edge_index)
    x = F.relu(x)
    x = self.conv2(x, edge_index)

    # 2. Pooling (把所有原子特徵都壓縮成一個分子的特徵)
    x = global_mean_pool(x, batch)

    # 3. 全連接層輸出
    x = self.fc(x)
    return x

# 初始化GNN
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(num_node_features =7).to(device)


#Training Phase: Baseline Model

In [ ]:
from matplotlib import use
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,roc_auc_score, classification_report
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

# 1. Initiate the model
rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# 2. Train the model
print("Training Random Forest...")
rf_model.fit(X_train, y_train)

print("Training XGBoost...")
xgb_model.fit(X_train, y_train)

# 3. Predict the result (probabilities and category)
rf_probs = rf_model.predict_proba(X_test)[:,1]
xgb_probs = xgb_model.predict_proba(X_test)[:,1]

rf_preds = rf_model.predict(X_test)
xgb_preds = xgb_model.predict(X_test)

# 4. 效能報告
def print_performance(name, y_true, y_pred, y_prob):
    print(f"\n===== {name} 效能報告 =====")
    print(f"AUC-ROC Score: {roc_auc_score(y_true, y_prob):.4f}")
    print(f"F1 Score:      {f1_score(y_true, y_pred):.4f}")
    print("\n詳細分類指標:")
    print(classification_report(y_true, y_pred))

print_performance("Random Forest", y_test, rf_preds, rf_probs)
print_performance("XGBoost", y_test, xgb_preds, xgb_probs)

Training Random Forest...
Training XGBoost...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [02:53:48] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



===== Random Forest 效能報告 =====
AUC-ROC Score: 0.9178
F1 Score:      0.8495

詳細分類指標:
              precision    recall  f1-score   support

           0       0.81      0.81      0.81       210
           1       0.85      0.85      0.85       263

    accuracy                           0.83       473
   macro avg       0.83      0.83      0.83       473
weighted avg       0.83      0.83      0.83       473


===== XGBoost 效能報告 =====
AUC-ROC Score: 0.9178
F1 Score:      0.8637

詳細分類指標:
              precision    recall  f1-score   support

           0       0.82      0.84      0.83       210
           1       0.87      0.86      0.86       263

    accuracy                           0.85       473
   macro avg       0.85      0.85      0.85       473
weighted avg       0.85      0.85      0.85       473



In [ ]:
# 生成MPH的Fingerprint
from rdkit.Chem import rdFingerprintGenerator

mp_smiles = "COC(=O)C(c1ccccc1)C1CCCCN1"

# 建立指紋生成器
mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)

# 轉換為分子物件
mol = Chem.MolFromSmiles(mp_smiles)

if mol:
  # 生成指紋物件
  fp = mfpgen.GetFingerprint(mol)

  mp_fp_array = np.zeros((1,), dtype = int)
  Chem.DataStructs.ConvertToNumpyArray(fp,mp_fp_array)

  print("指紋生成Success")
  print(f"指紋形狀(Shape):{mp_fp_array.shape}")

  # 看看哪些Bit是1(Active) - np.where 會回傳陣列中數值為1的index
  on_bits = np.where(mp_fp_array ==1)[0]
  print(f"亮起的Bits 數量:{len(on_bits)}")
  print(f"前 10 個亮起的 Bit 索引: {on_bits[:10]}")

  # 準備丟入模型預測 (Reshape 成 2D 矩陣)
  # 模型通常預期輸入形狀為 (樣本數, 特徵數)，所以要變成 (1, 2048)
  mp_input = mp_fp_array.reshape(1,-1)

else:
  print("❌ SMILES 解析失敗")



指紋生成Success
指紋形狀(Shape):(2048,)
亮起的Bits 數量:36
前 10 個亮起的 Bit 索引: [  1  54 147 185 305 325 389 604 650 695]


In [ ]:
# 丟進RF和XGB做預測

probability_rf = rf_model.predict_proba(mp_input)[0][1]
print(f"RF模型預測利他能對DAT有效的機率: {probability_rf:.4f}")

if probability_rf > 0.5:
  print("RF模型預測為有效")
else:
  print("RF模型預測為無效")

probability_xg = xgb_model.predict_proba(mp_input)[0][1]
print(f"XGB模型預測利他能對DAT有效的機率: {probability_xg:.4f}")

if probability_xg > 0.5:
  print("XGB模型預測為有效")
else:
  print("XGB模型預測為無效")

RF模型預測利他能對DAT有效的機率: 0.6167
RF模型預測為有效
XGB模型預測利他能對DAT有效的機率: 0.6489
XGB模型預測為有效


In [ ]:
# 1. 設定訓練參數
# =======================
# 若有GPU則用GPU; 否則用CPU

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用裝置:{device}")

# 初始化模型：num_node_features = 7
model = GCN(num_node_features = 7).to(device)

# 定義優化器(Optimiser) -使用Adam: 動量和RMSProp的優化器;機器學習中用於調整模型參數以最小化損失函數
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001, weight_decay = 5e-4)

# 定義Loss Function
criterion = torch.nn.CrossEntropyLoss()

# 2. 定義訓練函數
# ======================
def train():
  # 訓練mode
  model.train()
  total_loss = 0
  for batch in train_loader:
    batch = batch.to(device)

    # 清空梯度
    optimizer.zero_grad()

    # Forward Pass
    out = model(batch)

    # 計算Loss
    loss = criterion(out, batch.y)

    # Backward Pass
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  return total_loss / len(train_loader)

# 3. 定義測試/評估函數
# ======================

def test(loader):
  # 評估模式
  model.eval()

  y_true = []
  y_probs = []
  y_preds = []

  # 不計算梯度，節省記憶體
  with torch.no_grad():
    for batch in loader:
      batch = batch.to(device)
      out = model(batch)

      # 取得預測機率(Softmax)
      prob = F.softmax(out, dim=1)

      # 收集結果
      y_true.extend(batch.y.cpu().numpy())
      # 取Class 1 (Active)的機率
      y_probs.extend(prob[:,1].cpu().numpy())
      # 取機率最大的類別
      y_preds.extend(out.argmax(dim=1).cpu().numpy())

  # 計算指標
  auc = roc_auc_score(y_true, y_probs)
  f1 = f1_score(y_true, y_preds)
  return auc, f1

# 4. 開始訓練迴圈
# ======================
print("\n開始訓練ＧＮＮ")
# 訓練輪數
epochs = 50

for epoch in range(1, epochs+1):
  train_loss = train()

  # 每五個epoch檢查一次training set's efficacy

  if epoch % 5 == 0:
    train_auc, train_f1 = test(train_loader)
    test_auc, test_f1 = test(test_loader)

    print("Epoch: {:03d}, Loss: {:.4f}, Train AUC: {:.4f}, Test AUC: {:.4f}".format(
    epoch, train_loss, train_auc, test_auc))

print("\n訓練完成")


使用裝置:cpu

開始訓練ＧＮＮ
Epoch: 005, Loss: 0.6799, Train AUC: 0.6069, Test AUC: 0.6279
Epoch: 010, Loss: 0.6704, Train AUC: 0.6343, Test AUC: 0.6678
Epoch: 015, Loss: 0.6624, Train AUC: 0.6604, Test AUC: 0.7046
Epoch: 020, Loss: 0.6550, Train AUC: 0.6780, Test AUC: 0.7234
Epoch: 025, Loss: 0.6375, Train AUC: 0.7030, Test AUC: 0.7491
Epoch: 030, Loss: 0.6290, Train AUC: 0.7270, Test AUC: 0.7667
Epoch: 035, Loss: 0.6155, Train AUC: 0.7423, Test AUC: 0.7799
Epoch: 040, Loss: 0.6097, Train AUC: 0.7531, Test AUC: 0.7833
Epoch: 045, Loss: 0.5868, Train AUC: 0.7610, Test AUC: 0.7873
Epoch: 050, Loss: 0.5865, Train AUC: 0.7666, Test AUC: 0.7895

訓練完成


In [ ]:
# 最終效能評估
final_auc, final_f1 = test(test_loader)
print(f"\n=======GNN最終效能")
print(f"Test AUC-ROC: {final_auc:.4f}")
print(f"Test F1 Score: {final_f1:.4f}")

# If Results for GNN >0.7 --> 分子圖結構特徵確實捕捉到Fingerprints漏掉的資訊
if final_auc > 0.65:
  print("What an amazing progress! GNN's performance is better than the baseline so far")
else:
  print("GNN's performance is same or worse.")


=======GNN最終效能
Test AUC-ROC: 0.7895
Test F1 Score: 0.7765
What an amazing progress! GNN's performance is better than the baseline so far


#模型穩定性檢查 K-Fold Cross-Validation

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

def check_model_stability(model, X, y, n_splits=5):
  # 使用Stratified K-Fold to check model stability

  """
  Args:
    model: 定義好的模型物件(Random Forest/XGBoost)

    X: 特徵矩陣(Features)
    y: 標籤向量(Labels)
    n_splits: 折數(預設5)

  Returns:
    None
  """

  print(f"正在執行{n_splits}-Fold Cross-Validation以評估穩定性")

  # 使用StratifiedKFold確保每一折的類別比例一致
  cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

  # 計算分數
  scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
  print("-" * 40)
  print(f"各折分數(Scores per fold):{np.round(scores,4)}")
  print(f"平均準確率 (Mean Accuracy): {scores.mean():.4f}")
  print(f"標準差 (Standard Deviation): {scores.std():.4f}")
  print("-" * 40)

  # 判斷穩定性
  if scores.std() < 0.05:
    print("模型穩定性良好: 標準差<0.05")
  else:
    print("模型波動較大，建議調整數據量或參數")


✈

In [ ]:
print("RF model 穩定性：\n")
check_model_stability(rf_model, X_train, y_train)
print("\nXGB model 穩定性：")
check_model_stability(xgb_model, X_train, y_train)

RF model 穩定性：

正在執行5-Fold Cross-Validation以評估穩定性
----------------------------------------
各折分數(Scores per fold):[0.8571 0.8783 0.8598 0.8647 0.8488]
平均準確率 (Mean Accuracy): 0.8618
標準差 (Standard Deviation): 0.0098
----------------------------------------
模型穩定性良好: 標準差<0.05

XGB model 穩定性：
正在執行5-Fold Cross-Validation以評估穩定性


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [02:54:36] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [02:54:37] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [02:54:38] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [02:54:39] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [02:54:40] WARNING: /w

----------------------------------------
各折分數(Scores per fold):[0.873  0.8757 0.873  0.8568 0.8594]
平均準確率 (Mean Accuracy): 0.8676
標準差 (Standard Deviation): 0.0078
----------------------------------------
模型穩定性良好: 標準差<0.05


In [ ]:
from sklearn.model_selection import KFold
import torch
from torch_geometric.loader import DataLoader

# 1. 設定參數與準備
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)
results = [] # 用於記錄每一 fold 的結果

print(f"開始 {k_folds}-Fold 交叉驗證")

# 2. 開始切分資料
# dataset 是你原始的 PyG Dataset 物件
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    print(f"\n--- Fold {fold + 1} ---")

    # 根據索引建立該 Fold 的 Subset
    train_sub = [dataset[i] for i in train_ids]
    test_sub = [dataset[i] for i in test_ids]

    # 建立該 Fold 專用的 DataLoader
    train_loader = DataLoader(train_sub, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_sub, batch_size=32, shuffle=False)

    # 每個 Fold 都要「重新初始化」模型與優化器，確保不互相污染
    model = GCN(num_node_features=7).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
    criterion = torch.nn.CrossEntropyLoss()

    # 3. 訓練循環 (簡化為 20 epochs 作示範)
    for epoch in range(1, 21):
        # 這裡調用你之前寫好的 train() 函數
        # 注意：train() 內部的 loader 需改為當前的 train_loader
        loss = train_with_loader(model, train_loader, optimizer, criterion)

    # 4. 評估模型
    # 調用你之前的 test() 函數取得該 fold 的表現
    auc, f1 = test_with_loader(model, test_loader)
    print(f"Fold {fold + 1} 結果: AUC: {auc:.4f}, F1: {f1:.4f}")

    results.append((auc, f1))

# 5. 計算平均表現
avg_auc = sum(r[0] for r in results) / k_folds
avg_f1 = sum(r[1] for r in results) / k_folds
print(f"\n最終平均表現: AUC: {avg_auc:.4f}, F1: {avg_f1:.4f}")